In [1]:
# Import our python files
import categorize_relevance, find_questions, preprocessing
# Note that this might take some time to load

2022-09-30 11:32:48.293694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-30 11:32:48.293769: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
# For demonstration purposes, we will use one of our conversations from our dataset
transcription_file = "../data/example_conversation.txt"
wiki_tfidf_file = "../data/wiki_tfidf_terms.csv"

In [2]:
# Preprocess our text file into speakerturns
speakerturns = preprocessing.preprocessing(transcription_file)
# Turn our speakerturns into a dataframe, so we can label our data
df = preprocessing.create_labeled_dataframe(speakerturns)

In [3]:
# This is what our data currently looks like
df.head()

,identifier,time,speaker,text,question,relevant
identifier,,,,,,
0,0,[0:00:00],spk_0,we're doing I'm *** and this is **** we are co...,0,0
1,1,[0:01:17],spk_1,Expense category. So the budget consists of in...,0,0
2,2,[0:02:15],spk_2,"Okay, so the IFA is like an association of dif...",0,0
3,3,[0:02:25],spk_1,No no no no no no. Okay. So the idea is that e...,0,0
4,4,[0:02:43],spk_0,"Sorry, sorry to interrupt.",0,0


In [5]:
# Finding our questions, using Dialog Acts in our case
df_q = find_questions.dialog_act_questions(df)
# This might take a while...
# Alternatively one could use POS tags:
# df_q = find_questions.pos_questions(df) # Note that this requires StanfordCoreNLP to be ran on localhost

bert-base-uncased found in cache. Loading model...


2022-09-30 11:34:25.052103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-30 11:34:25.052580: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-30 11:34:25.052647: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-139JLDD): /proc/driver/nvidia/version does not exist
2022-09-30 11:34:25.054772: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of

In [8]:
# Now our data looks like this, when looking at some of the questions
df_q[df_q.question == 1].head()

,identifier,time,speaker,text,question,relevant
identifier,,,,,,
0,0,[0:00:00],spk_0,we're doing I'm *** and this is **** we are co...,1,0
1,1,[0:01:17],spk_1,Expense category. So the budget consists of in...,1,0
2,2,[0:02:15],spk_2,"Okay, so the IFA is like an association of dif...",1,0
12,12,[0:03:51],spk_0,Okay. All right. Um and um can you uh give us ...,1,0
13,13,[0:04:11],spk_1,for the budgeting system? I think the IFA admi...,1,0


In [12]:
# Now, we will create our relevance indicating words list from our original transcription file
# Here, we take 60 of the words after which we filter them, using our wikipedia tfidf file
relevance_words = categorize_relevance.tfidf_word_list(transcription_file, 60, wiki_tfidf_file)
# This will take a while, as we process a big TF-IDF file from the wikipedia dump

 52%|█████▏    | 5381653/10394041 [00:27<00:24, 201406.98it/s]

Error on: "−7,3,13,23..",1,5989879,15.605581769553712



 59%|█████▉    | 6173449/10394041 [00:32<00:19, 215681.93it/s]

Error on: "2,4,6,8..",1,5989879,15.605581769553712



 73%|███████▎  | 7546001/10394041 [00:39<00:13, 210333.76it/s]

Error on: "১,৭৫,৫৭,৩৪৫",1,5989879,15.605581769553712



 79%|███████▉  | 8256742/10394041 [00:42<00:11, 184680.09it/s]

Error on: ",3f.d.m.,3s.h.p.d.,8s.h.p.d",1,5989879,15.605581769553712



10394040it [00:54, 192086.17it/s]1 [00:54<00:00, 186540.42it/s]
100%|█████████▉| 10394040/10394041 [00:54<00:00, 192073.27it/s]
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 10099022/10099022 [02:23<00:00, 70152.18it/s]


Keywords of transcript according to wiki TF-IDF ['ifa', 'scheduling', 'referees', 'budgeting', 'referee', 'notifications', 'notification', 'portal', 'manually', 'data', 'scheduler', 'budget', 'schedule', 'say', 'kind', 'faa', 'explicate', 'insert', 'system', 'policy', 'reporting', 'update', 'expect', 'expecting', 'game', 'teams', 'example', 'players', 'understand', 'question', 'communication']


In [13]:
# Finally, we filter out our questions using these relevance indicating words
df_rq = categorize_relevance.filter_questions(df_q, relevance_words)

In [14]:
# Our relevant questions finally look like this:
df_rq[df_rq.relevant == 1].head()

,identifier,time,speaker,text,question,relevant
identifier,,,,,,
0,0,[0:00:00],spk_0,we're doing I'm *** and this is **** we are co...,1,1
1,1,[0:01:17],spk_1,Expense category. So the budget consists of in...,1,1
2,2,[0:02:15],spk_2,"Okay, so the IFA is like an association of dif...",1,1
12,12,[0:03:51],spk_0,Okay. All right. Um and um can you uh give us ...,1,1
13,13,[0:04:11],spk_1,for the budgeting system? I think the IFA admi...,1,1
